In [1]:
using PauliPropagation
using Random
using Optim
using Plots
Random.seed!(43)
using ReverseDiff: GradientTape, gradient!, compile, gradient
using LinearAlgebra
using StatsBase 
using GLM
using DataFrames
using CSV

# CDR for quantum dynamics
- TFIM Hamiltonian (constants not site-dependent) evolving with TDSE
- discretize the evolution steps and use first-order Trotter decomposition 

### Idea:
- DONE: Run "exact" evolution for small trotterized circuit
- DONE: Create near-Clifford circuits:replace some gates in Trotterized gate by close Cliffords - no VQA here so this is much simpler, find closest Clifford to the RZZ and to the RX (only have 2 angles in total), replace a portion randomly (no MCMC), keep the parameter of N = non-Cliffords const. st. system remains cl. scalable.
- DONE: create MWE of CDR within quantum dynamics
- which gates ones we replace has influence on the accuracy of the expectation value (principle of causal light cone as seen in vnCDR paper Piotr, is this automatically respected within Heisenberg picture? (backprop observable)), so we can aim to replace only gates that contribute to an expectation value (works only for single-qubit / local observables)
- another idea is to try the perturbation approach (all non-Cliffords) 

In [2]:
struct trotter_ansatz
    target_circuit::Vector{Gate}
    topology::Vector{Tuple{Int64, Int64}}
    nqubits::Integer
    steps::Integer #layers
    time::Integer
    J::Float64
    h::Float64
    sigma_J::Float64
    sigma_h::Float64
    sigma_J_indices::Vector{Int64}
    sigma_h_indices::Vector{Int64}
end

In [3]:
function trotter_setup(nqubits::Integer, steps::Integer, time::Float64, J::Float64, h::Float64;topology = nothing)
    if isnothing(topology)
        topology = bricklayertopology(nqubits)
    end
    target_circuit = tfitrottercircuit(nqubits,steps,topology=topology) #start with RZZ layer
   
    sigma_J = -2*T*J/steps
    sigma_h = 2*T*h/steps 

    sigma_J_indices = getparameterindices(target_circuit, PauliRotation, [:Z,:Z]) 
    sigma_h_indices = getparameterindices(target_circuit, PauliRotation, [:X])
    
    return trotter_ansatz(target_circuit, topology, nqubits, steps, time, J, h,sigma_J, sigma_h,sigma_J_indices, sigma_h_indices)
end

trotter_setup (generic function with 1 method)

In [4]:
function constrain_params(ansatz)
    nparams = countparameters(ansatz.target_circuit)
    thetas = zeros(nparams)
    thetas[ansatz.sigma_h_indices] .= ansatz.sigma_h
    thetas[ansatz.sigma_J_indices] .= ansatz.sigma_J
    return thetas
end

constrain_params (generic function with 1 method)

In [5]:
function obs_magnetization(ansatz)
    magnetization = PauliSum(ansatz.nqubits)
    for i in 1:nq
        add!(magnetization,:Z,i)
    end
    magnetization = magnetization/nq
    return magnetization
end

obs_magnetization (generic function with 1 method)

In [6]:
function exact_trotter_time_evolution(ansatz)
    thetas = constrain_params(ansatz)
    obs = obs_magnetization(ansatz)
    circuit = copy(ansatz.target_circuit)
    psum =propagate(circuit,obs, thetas)
    return overlapwithzero(psum)
end

exact_trotter_time_evolution (generic function with 1 method)

In [7]:
function training_set_generation(ansatz::trotter_ansatz; num_samples::Int = 10, non_cliffs::Int = 30)
    nparams = countparameters(ansatz.target_circuit)
    cliffs = nparams - non_cliffs

    ratio = length(ansatz.sigma_J_indices)/(length(ansatz.sigma_h_indices) + length(ansatz.sigma_J_indices))
    num_h = Int(round((1-ratio)*cliffs))
    num_J = Int(round(ratio*cliffs))
    println("Number of H terms: ", num_h)
    println("Number of J terms: ", num_J)

    training_thetas_list = Vector{Vector{Float64}}()
    thetas = constrain_params(ansatz)
    for _ in 1:num_samples
        training_thetas = deepcopy(thetas)
        shuffled_sigma_h_indices =  Random.shuffle!(ansatz.sigma_h_indices)
        shuffled_sigma_J_indices = Random.shuffle!(ansatz.sigma_J_indices)
        selected_indices_h = shuffled_sigma_h_indices[1:num_h]
        selected_indices_J = shuffled_sigma_J_indices[1:num_J];   
        k_h = round(ansatz.sigma_h/(π/4))
        k_J = round(ansatz.sigma_J/(π/4))

        for i in selected_indices_h
            training_thetas[i] = k_h*π/4
        end
        for i in selected_indices_J
            training_thetas[i] = k_J*π/4
        end
        push!(training_thetas_list, training_thetas)
    end
    return training_thetas_list
end

training_set_generation (generic function with 1 method)

In [8]:
function exact_time_evolution(ansatz,training_thetas; noisy_circuit = false)
    
    if noisy_circuit == false
        noisy_circuit = copy(ansatz.target_circuit)
    end

    exact_expvals = Vector{Float64}()
    for thetas in training_thetas
        obs = obs_magnetization(ansatz)
        psum =propagate(noisy_circuit,obs, thetas)
        push!(exact_expvals, overlapwithzero(psum))
    end
    return exact_expvals
end

exact_time_evolution (generic function with 1 method)

In [9]:
function final_noise_layer_circuit(ansatz; depol_strength = 0.05, dephase_strength = 0.05)
        #to be replaced with a decent noise model
        depol_noise_layer = [DepolarizingNoise(qind, depol_strength ) for qind in 1:ansatz.nqubits];
        dephase_noise_layer = [DephasingNoise(qind, dephase_strength) for qind in 1:ansatz.nqubits];
        noisy_circuit = deepcopy(ansatz.target_circuit)
        append!(noisy_circuit,depol_noise_layer)
        append!(noisy_circuit,dephase_noise_layer)

        return noisy_circuit
end

final_noise_layer_circuit (generic function with 1 method)

In [10]:
function gate_noise_circuit(ansatz;depol_strength =0.01, dephase_strength = 0.01, topology=nothing, start_with_ZZ=true)
    circuit::Vector{Gate} = []

    if isnothing(topology)
        topology = bricklayertopology(ansatz.nqubits)
    end

    # the function after this expects a circuit with at least one layer and will always append something
    if ansatz.steps == 0
        return circuit
    end

    depol_noise_layer = [DepolarizingNoise(qind, depol_strength ) for qind in 1:ansatz.nqubits];
    phase_damp_layer = [DephasingNoise(qind, dephase_strength) for qind in 1:ansatz.nqubits];

    if start_with_ZZ
        rzzlayer!(circuit, ansatz.topology)
        append!(circuit,depol_noise_layer)
        append!(circuit,phase_damp_layer)
    end

    for _ in 1:ansatz.steps-1
        rxlayer!(circuit, ansatz.nqubits)
        append!(circuit,depol_noise_layer)
        append!(circuit,phase_damp_layer)
        rzzlayer!(circuit, ansatz.topology)
        append!(circuit,depol_noise_layer)
        append!(circuit,phase_damp_layer)
    end

    rxlayer!(circuit, ansatz.nqubits)
    append!(circuit,depol_noise_layer)
    append!(circuit,phase_damp_layer)

    if !start_with_ZZ
        rzzlayer!(circuit, ansatz.topology)
        append!(circuit,depol_noise_layer)
        append!(circuit,phase_damp_layer)
    end

    return circuit
end

gate_noise_circuit (generic function with 1 method)

In [11]:
function cdr(noisy_exp_values::Vector{Float64}, exact_exp_values::Vector{Float64}, noisy_target_exp_value::Float64, exact_target_exp_value::Float64; verbose=false)
    training_data = DataFrame(x=noisy_exp_values,y=exact_exp_values)
    ols = lm(@formula(y ~ x), training_data)
    function cdr_em(x)
        return  coef(ols)[1] + coef(ols)[2] * x
    end
    rel_error_after = abs(exact_target_exp_value - cdr_em(noisy_target_exp_value)) / abs(exact_target_exp_value)
    rel_error_before = abs(exact_target_exp_value - noisy_target_exp_value) / abs(exact_target_exp_value)
    if verbose
        println(training_data)
        println("Noisy target expectation value: ", noisy_target_exp_value)
        println("Relative error before CDR: ", rel_error_before)
        println("CDR-EM target expectation value: ", cdr_em(noisy_target_exp_value))
        println("Relative error after CDR: ", rel_error_after)
    end
    return cdr_em(noisy_target_exp_value), rel_error_after, rel_error_before
end 

cdr (generic function with 1 method)

## MWE
### Exact evolution of a small trotterized circuit (see CPDR p.7)

In [12]:
nq = 4
heaxy_hex_2_topology = [(1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), 
(10, 11), (11, 12), (12, 1), (5, 13), (13, 14), (14, 15), (15, 16), 
(16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 7)]
steps = 10
T = 1.0
J = 5.0 # J > 0 in ferromagnetic phase, J < 0 in antiferromagnetic phase
h = 1.0 #abs(h) < abs(J) in ordered phase
trotter = trotter_setup(nq, steps, T, J, h);
noisy_circuit = gate_noise_circuit(trotter, depol_strength = 0.01, dephase_strength = 0.01)

230-element Vector{Gate}:
 PauliRotation([:Z, :Z], [1, 2])
 PauliRotation([:Z, :Z], [3, 4])
 PauliRotation([:Z, :Z], [2, 3])
 FrozenGate(DepolarizingNoise(1), parameter = 0.01)
 FrozenGate(DepolarizingNoise(2), parameter = 0.01)
 FrozenGate(DepolarizingNoise(3), parameter = 0.01)
 FrozenGate(DepolarizingNoise(4), parameter = 0.01)
 FrozenGate(DephasingNoise(1), parameter = 0.01)
 FrozenGate(DephasingNoise(2), parameter = 0.01)
 FrozenGate(DephasingNoise(3), parameter = 0.01)
 ⋮
 PauliRotation([:X], [4])
 FrozenGate(DepolarizingNoise(1), parameter = 0.01)
 FrozenGate(DepolarizingNoise(2), parameter = 0.01)
 FrozenGate(DepolarizingNoise(3), parameter = 0.01)
 FrozenGate(DepolarizingNoise(4), parameter = 0.01)
 FrozenGate(DephasingNoise(1), parameter = 0.01)
 FrozenGate(DephasingNoise(2), parameter = 0.01)
 FrozenGate(DephasingNoise(3), parameter = 0.01)
 FrozenGate(DephasingNoise(4), parameter = 0.01)

In [13]:
exact_expval_target = exact_trotter_time_evolution(trotter) #should be close to one as we stay in FM phase

0.9473863474250936

In [14]:
noisy_expval_target = exact_time_evolution(trotter, [constrain_params(trotter)]; noisy_circuit = noisy_circuit)[1]

0.7369338158668288

In [15]:
num_samples = 10
list = training_set_generation(trotter; num_samples = 10, non_cliffs = 30);

Number of H terms: 23
Number of J terms: 17


In [16]:
exact_expvals = exact_time_evolution(trotter,list)

10-element Vector{Float64}:
 0.9064175420081971
 0.9681421490055432
 0.9629855018934352
 0.9427600849618765
 0.9521035580443389
 0.9421581470310213
 0.8684598296727593
 0.9179467287175591
 0.9694203780675554
 0.9153216275620748

In [17]:
noisy_expvals = exact_time_evolution(trotter,list; noisy_circuit = noisy_circuit)

10-element Vector{Float64}:
 0.7358254862658001
 0.7844802852532986
 0.7776700853529712
 0.7642582037152266
 0.7661752015609321
 0.7642089163265727
 0.7116460149680841
 0.7432122314503923
 0.7815892834658476
 0.7378423054184801

In [18]:
noisy_circ = gate_noise_circuit(trotter, depol_strength = 0.01, dephase_strength = 0.01);

In [19]:
corr_energy, rel_error_after, rel_error_before = cdr(noisy_expvals, exact_expvals, noisy_expval_target[1], exact_expval_target; verbose=true)  

10×2 DataFrame
 Row │ x         y        
     │ Float64   Float64  
─────┼────────────────────
   1 │ 0.735825  0.906418
   2 │ 0.78448   0.968142
   3 │ 0.77767   0.962986
   4 │ 0.764258  0.94276
   5 │ 0.766175  0.952104
   6 │ 0.764209  0.942158
   7 │ 0.711646  0.86846
   8 │ 0.743212  0.917947
   9 │ 0.781589  0.96942
  10 │ 0.737842  0.915322
Noisy target expectation value: 0.7369338158668288
Relative error before CDR: 0.22214013546876082
CDR-EM target expectation value: 0.9077668990953871
Relative error after CDR: 0.041819737467600676


(0.9077668990953871, 0.041819737467600676, 0.22214013546876082)

## Next steps:
- DONE: make realistic noise model
- change observable to where correction is not as easy
- other breaking points: resolution of the timesteps, i.e. smaller gate angels (wont work with Clifford replacement) 